In [ ]:
#TODO: Can I put this in another file and import it somehow?
from pynq import DefaultIP
from matplotlib import pyplot as plt
from scipy.stats import sem

class ROControlDriver(DefaultIP):
    def __init__(self, description):
        super().__init__(description=description)
        self.rm = self.register_map
    
    bindto = ['wiresboy.github.io:Brandon:axi_ro_control:1.0.2']
    
    def reset_ro(self):
        self.rm.Control[0] = 1
    
    def start_acquire(self):
        self.rm.Control[1] = 1
    
    @property
    def ro_quantity(self):# ROs enabled
        return self.rm.ro_quantity[31:16]
    @ro_quantity.setter
    def ro_quantity(self, num_ro_enabled):
        self.rm.RO_Quantity_Control[31:16] = num_ro_enabled
        
    
    @property
    def cycles(self): # cycles/integration
        return self.rm.RO_Quantity_Control[15:0]
    @cycles.setter
    def cycles(self, cycles_per_integration):
        self.rm.RO_Quantity_Control[15:0] = cycles_per_integration
    
    @property
    def power_virus(self): # of PV enabled
        return self.rm.Power_Virus_Control
    @power_virus.setter
    def power_virus(self, num_power_virus):
        self.rm.Power_Virus_Control = num_power_virus

    
    @property
    def mode(self): # mode: 1=continuous, 0=one shot
        return self.rm.Mode[0]
    @mode.setter
    def mode(self, mode):
        self.rm.Mode[0] = mode
        
    @property
    def status(self): #Status byte
        return self.rm.Control[7:0]  #Some bits of current index into bram
    
    @property
    def last_read_sum(self):
        return self.rm.Control[31:16] 
    


class XADCDriver(DefaultIP):
    def __init__(self, description):
        super().__init__(description=description)
    
    bindto = ['xilinx.com:ip:xadc_wiz:3.3']

    @property
    def temperature_raw(self):
        return self.read(0x200) >> 4
    @property
    def temperature_c(self):
        return (self.temperature_raw * 503.975/4096)-273.15
    @property
    def temperature_f(self):
        return ((self.temperature_raw * 503.975/4096)-273.15)*9/5 +32
    
    @property
    def vccint_raw(self):
        return self.read(0x204) >> 4
    @property
    def vccint(self):
        return self.vccint_raw *3/4096
    
    @property
    def vccaux_raw(self):
        return self.read(0x208) >> 4
    @property
    def vccaux(self):
        return self.vccaux_raw *3/4096
    
    @property
    def vccbram_raw(self):
        return self.read(0x218) >> 4
    @property
    def vccbram(self):
        return self.vccbram_raw *3/4096
    
    @property
    def vccpint_raw(self):
        return self.read(0x234) >> 4
    @property
    def vccpint(self):
        return self.vccpint_raw *3/4096


def get_bram():
    return [bram.read(offset=x) for x in range(0,bram.length, 4)]


#Return: list of values
def run_test(ro_quantity=1, ro_cycles = 2, power_virus = 0):
    ro.ro_quantity = ro_quantity
    ro.cycles = ro_cycles
    ro.power_virus = power_virus
    ro.mode = 0 # One-shot
    ro.reset_ro()
    ro.start_acquire()
    while (ro.status < 0xff): #Wait for acquisition to finish. 
        #TODO: How to tell when long-duration tests are done? This doesn't show all the bits...
        pass
    return get_bram()

In [ ]:
from pynq import Overlay
overlay = Overlay("z2_voltage_experiments_wrapper.bit")

ro = overlay.axi_ro_control_0
xadc=overlay.xadc_wiz_0
bram = overlay.axi_bram_ctrl_0.mmio

In [ ]:
ROQ = 256
ROC = 40 

xlist = []
ylist = []
adclist = []
yerrlist = []
adcerrlist = []
for pv in range(0, (2**16)-1, 2047):
    sample = run_test(ro_quantity=ROQ, ro_cycles = ROC, power_virus = pv)
    adcsample = []
    for x in range(200): #Get 2000 ADC readings
        adcsample.append(xadc.vccpint)
    average = sum(sample)/len(sample)
    stderr = sem(sample)
    xlist.append(pv)
    ylist.append(average)
    yerrlist.append(stderr)
    
    adclist.append(sum(adcsample)/len(adcsample))
    adcerrlist.append(sem(adcsample))



In [ ]:
fig, (plt_ro, plt_adc) = plt.subplots(2, sharex=True)
fig.suptitle("RO Quantity="+str(ROQ)+", Cycles="+str(ROC))
fig.subplots_adjust(left=0.15)

plt_ro.errorbar(xlist, ylist, yerr=yerrlist)
plt_ro.scatter(xlist, ylist)
plt_ro.set(ylabel="RO Count")
plt_ro.set_title("RO sum vs #PV (RO)")

plt_adc.errorbar(xlist, adclist, yerr=adcerrlist)
plt_adc.scatter(xlist, adclist)
plt_adc.set(xlabel="Number of Power Virus")
plt_adc.set(ylabel="ADC: VCCINT voltage")
plt_adc.set_title("Measured ADC vs #PV (RO)")

fig.savefig("ROQuantity"+str(ROQ)+" Cycles"+str(ROC)+".png", dpi=300)

In [ ]:
for cyc in [1,2,3,10,100,200,500]:
    print("\n  ",cyc)
    ro.ro_quantity = 4 # ROs enabled
    ro.cycles = cyc # # cycles/integration
    ro.power_virus = 0x11
    ro.mode = 1 #Continuous
    ro.reset_ro()
    ro.start_acquire()
    from time import sleep
    for x in range(4):
        print(ro.status, "\t", ro.last_read_sum)

In [ ]:
#SW power virus
ro.ro_quantity = 5
zz = 1;
for x in range(1,100000):
    zz = zz*x